<a href="https://colab.research.google.com/github/marcos-da-luz-pinto/TSFMs/blob/main/Moirai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/SalesforceAIResearch/uni2ts.git
!pip install pandas numpy torch scipy

  Cloning https://github.com/SalesforceAIResearch/uni2ts.git to /tmp/pip-req-build-1slascxc
  Running command git clone --filter=blob:none --quiet https://github.com/SalesforceAIResearch/uni2ts.git /tmp/pip-req-build-1slascxc
  Resolved https://github.com/SalesforceAIResearch/uni2ts.git to commit 9a78b80c1194bb86db2c0fa4149f46df255a1627
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
!pip install --upgrade pip

!pip uninstall -y torch torchvision torchaudio

!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121

!pip install git+https://github.com/SalesforceAIResearch/uni2ts.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Found existing installation: torch 2.4.1
Uninstalling torch-2.4.1:
  Successfully uninstalled torch-2.4.1
Found existing installation: torchvision 0.24.0+cpu
Uninstalling torchvision-0.24.0+cpu:
  Successfully uninstalled torchvision-0.24.0+cpu
Found existing installation: torchaudio 2.9.0+cpu
Uninstalling torchaudio-2.9.0+cpu:
  Successfully uninstalled torchaudio-2.9.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 MB 42.7 MB/s  0:00:17
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 116.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 25.8 MB/s  0:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import numpy as np
import torch
import warnings

from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

warnings.filterwarnings("ignore")

# CARREGAMENTO DOS DADOS
df = pd.read_excel("data.xlsx")
df["Data"] = pd.to_datetime(df["Data"])
df = df.sort_values("Data").set_index("Data")

colunas_alvo = [
    "INCC- DI_var_mes", "Taxa_SELIC_avg", "CUB Global",
    "Cub Material", "CUB Mão de Obra",
    "CUB Desp. Administrativa", "CUB Equipamento"
]

contexts = [24, 60, 120]
horizons = [12, 24]
n_windows = 10
NUM_SAMPLES = 20

device = "cuda" if torch.cuda.is_available() else "cpu"

# MÉTRICAS
def calculate_mase(y_true, y_pred, y_train, seasonal_period=12):
    denom = np.mean(np.abs(np.diff(y_train, seasonal_period)))
    if denom == 0:
        return np.nan
    return np.mean(np.abs(y_true - y_pred)) / denom

def calculate_crps_samples(y_true, samples):
    y_true = np.array(y_true)
    term1 = np.mean(np.abs(samples - y_true), axis=0)
    s_i = samples[:, np.newaxis, :]
    s_j = samples[np.newaxis, :, :]
    term2 = np.mean(np.abs(s_i - s_j), axis=(0, 1))
    return np.mean(term1 - 0.5 * term2)

# MODELO MOIRAI
print("Carregando Moirai Base...")
module = MoiraiModule.from_pretrained("Salesforce/moirai-base")

# Atenção: context_length e prediction_length
# serão definidos dinamicamente
results_moirai = []

# LOOP EXPERIMENTAL (ROLLING ORIGIN)
for col in colunas_alvo:
    series = df[col].dropna()
    print(f"Executando Moirai: {col}")

    for C in contexts:
        for H in horizons:

            model = MoiraiForecast(
                module=module,
                context_length=C,
                prediction_length=H,
                patch_size=max(4, C // 10),
                num_samples=NUM_SAMPLES,
                target_dim=1,
                feat_dynamic_real_dim=0,
                past_feat_dynamic_real_dim=0,
            ).to(device)

            predictor = model.create_predictor(batch_size=1)

            mase_list, crps_list = [], []

            for w in range(n_windows):
                end_idx = len(series) - w
                start_idx = end_idx - (C + H)
                if start_idx < 0:
                    break

                train = series.iloc[start_idx:start_idx+C].values
                test = series.iloc[start_idx+C:end_idx].values

                try:
                    from gluonts.dataset.common import ListDataset

                    dataset = ListDataset(
                        [{
                            "start": series.index[start_idx],
                            "target": train
                        }],
                        freq="M"  # ou "MS" se seus dados forem início do mês
                    )

                    forecast = next(predictor.predict(dataset))
                    forecast_samples = forecast.samples  # shape: [num_samples, H]

                    forecast_median = np.median(forecast_samples, axis=0)

                    mase_list.append(
                        calculate_mase(test, forecast_median, train)
                    )
                    crps_list.append(
                        calculate_crps_samples(test, forecast_samples)
                    )

                except Exception as e:
                    continue

            if mase_list:
                results_moirai.append({
                    "Variável": col,
                    "C": C,
                    "H": H,
                    "MASE (Mediana)": round(np.median(mase_list), 4),
                    "CRPS (Mediana)": round(np.median(crps_list), 4),
                    "Modelo": "Moirai-Base"
                })

# RESULTADOS
results_df_moirai = pd.DataFrame(results_moirai)

print("\n" + "="*40)
print("RESULTADOS FINAIS - MOIRAI")
print("="*40)
print(results_df_moirai.to_string(index=False))

# Importar resultados em csv
results_df_moirai.to_csv("RESULTADOS_Moirai.csv", index=False)


Carregando Moirai Base...
Executando Moirai: INCC- DI_var_mes
Executando Moirai: Taxa_SELIC_avg
Executando Moirai: CUB Global
Executando Moirai: Cub Material
Executando Moirai: CUB Mão de Obra
Executando Moirai: CUB Desp. Administrativa
Executando Moirai: CUB Equipamento

RESULTADOS FINAIS - MOIRAI
                Variável   C  H  MASE (Mediana)  CRPS (Mediana)      Modelo
        INCC- DI_var_mes  24 12          0.0008          0.2806 Moirai-Base
        INCC- DI_var_mes  24 24          0.0030          0.7404 Moirai-Base
        INCC- DI_var_mes  60 12          0.0011          0.2986 Moirai-Base
        INCC- DI_var_mes  60 24          0.0017          0.3665 Moirai-Base
        INCC- DI_var_mes 120 12          0.0008          0.2698 Moirai-Base
        INCC- DI_var_mes 120 24          0.0009          0.3309 Moirai-Base
          Taxa_SELIC_avg  24 12          0.0285          1.2802 Moirai-Base
          Taxa_SELIC_avg  24 24          0.0252          2.9255 Moirai-Base
          Taxa_S